In [12]:
# Import Required Libraries
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Input, Dropout
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from datetime import datetime
import pickle

print("TensorFlow version:", tf.__version__)
print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)

TensorFlow version: 2.12.0
NumPy version: 1.23.5
Pandas version: 2.3.1


In [ ]:
# Load and Explore Data
df_b = pd.read_csv("") # train
df_d = pd.read_csv("") # validation"""
df_c = pd.read_csv("") # test

df_b = df_b[df_b['Timestamp_Local'].str.slice(11, 16).between('07:00', '19:00')]
df_d = df_d[df_d['Timestamp_Local'].str.slice(11, 16).between('07:00', '19:00')]
df_c = df_c[df_c['Timestamp_Local'].str.slice(11, 16).between('07:00', '19:00')]

print("Site B shape:", df_b.shape)
print("Site D shape:", df_d.shape)
print("Site C shape:", df_c.shape)

In [ ]:
def calculate_hourly_averages(df):
    """Calculate actual hourly average power from training data"""
    df_temp = df.copy()
    df_temp['Timestamp_Local'] = pd.to_datetime(df_temp['Timestamp_Local'])
    df_temp['hour'] = df_temp['Timestamp_Local'].dt.hour
    
    # Calculate hourly averages for business hours (10-18)
    business_hours = range(10, 19)  # 10 to 18 inclusive
    hourly_averages = {}
    
    print("Calculating hourly averages from Site B training data:")
    for hour in business_hours:
        hour_data = df_temp[df_temp['hour'] == hour]['Building_Power_kW']
        if len(hour_data) > 0:
            avg_power = hour_data.mean()
            hourly_averages[hour] = avg_power
            print(f"Hour {hour}: {avg_power:.2f} kW (from {len(hour_data)} samples)")
        else:
            print(f"Hour {hour}: No data available")
    
    return hourly_averages


def create_enhanced_features(df, hourly_avg_power=None):
    """
    Create time-based features and power deviation features from timestamp and building power columns
    Business hours considered 10:00-18:00 inclusive.
    """
    # Make a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Convert timestamp to datetime
    df['Timestamp_Local'] = pd.to_datetime(df['Timestamp_Local'])
    
    # Extract hour and month
    df['hour'] = df['Timestamp_Local'].dt.hour
    df['month'] = df['Timestamp_Local'].dt.month
    
    # === TIME-BASED FEATURES ===
    # Create is_10_to_18 feature (business hours: 10:00 AM - 6:00 PM inclusive)
    df['is_10_to_18'] = ((df['hour'] >= 10) & (df['hour'] <= 17)).astype(int)
    
    df['is_10'] = (df['hour'] == 10).astype(int)
    df['is_11'] = (df['hour'] == 11).astype(int)
    df['is_12'] = (df['hour'] == 12).astype(int)
    df['is_13'] = (df['hour'] == 13).astype(int)
    df['is_14'] = (df['hour'] == 14).astype(int)
    df['is_15'] = (df['hour'] == 15).astype(int)
    df['is_16'] = (df['hour'] == 16).astype(int)
    df['is_17'] = (df['hour'] == 17).astype(int)

    # Create monthly indicator features
    df['is_january'] = (df['month'] == 1).astype(int)
    df['is_february'] = (df['month'] == 2).astype(int)
    df['is_june'] = (df['month'] == 6).astype(int)
    df['is_july'] = (df['month'] == 7).astype(int)
    df['is_august'] = (df['month'] == 8).astype(int)
    df['is_december'] = (df['month'] == 12).astype(int)
    
    # === PATTERN + DEVIATION FEATURES ===
    # Hour-specific pattern flags (updated bounds)
    df['is_morning_peak'] = ((df['hour'] >= 10) & (df['hour'] <= 11)).astype(int)  # 10-12
    df['is_afternoon_low'] = ((df['hour'] >= 12) & (df['hour'] <= 17)).astype(int) # 13-18
    
    # Use calculated hourly averages if provided, otherwise fallback to overall mean
    if hourly_avg_power is None:
        print("Warning: Using overall mean as no hourly averages provided")
        df['expected_power'] = df['Building_Power_kW'].mean()
    else:
        # Power deviation from calculated hourly average
        df['expected_power'] = df['hour'].map(hourly_avg_power).fillna(df['Building_Power_kW'].mean())
    
    df['power_deviation'] = df['Building_Power_kW'] - df['expected_power']
    df['power_deviation_ratio'] = df['power_deviation'] / df['expected_power']
    
    # Drop the temporary expected_power column as we have the deviation features
    df = df.drop(columns=['expected_power'])
    
    return df

def create_learnable_signals(df, hourly_avg_power=None, seq_length=192):
    """
    Create learnable signal features for LSTM sequences (computed on-the-fly, not persisted).
    These are goal-aligned features that help the model learn the domain rules.
    Returns: df with additional computed columns for use in sequences
    """    
    df = df.copy()
    
    # === CYCLICAL TIME ENCODING ===
    # Hour cyclical (0-23 -> 0-2π)
    hour_rad = 2 * np.pi * df['hour'] / 24
    df['hour_sin'] = np.sin(hour_rad)
    df['hour_cos'] = np.cos(hour_rad)
    
    # Month cyclical (1-12 -> 0-2π)
    month_rad = 2 * np.pi * (df['month'] - 1) / 12
    df['month_sin'] = np.sin(month_rad)
    df['month_cos'] = np.cos(month_rad)
    
    # === SOFT MASKS ===
    # Business hours soft mask (already have is_10_to_18, but rename for clarity)
    df['is_in_business_hours'] = df['is_10_to_18']
    
    # Position within business hours window (0-1 scale)
    df['position_in_window'] = np.where(
        df['is_in_business_hours'] == 1,
        (df['hour'] - 10) / 8,  # 10->0, 18->1
        0
    )
    
    # Allowed month soft mask
    allowed_months = {1, 2, 6, 7, 8, 12}
    df['is_allowed_month'] = df['month'].isin(allowed_months).astype(int)
    
    # === CAUSAL POWER DELTAS ===
    power_col = 'Building_Power_kW'
    
    # Rolling deltas (causal - looking back only)
    # 15-minute delta (assuming 15-min intervals)
    df['trailing_15m_delta'] = df[power_col] - df[power_col].shift(1)
    
    # 30-minute delta (2 periods back)
    df['trailing_30m_delta'] = df[power_col] - df[power_col].shift(2)
    
    # 60-minute average (4 periods back, including current)
    df['trailing_60m_avg'] = df[power_col].rolling(window=4, min_periods=1).mean()
    
    # === RESIDUALS (EXPECTED VS ACTUAL) ===
    # Use calculated hourly expectations instead of hardcoded values
    if hourly_avg_power is None:
        print("Warning: Using overall mean for residuals as no hourly averages provided")
        expected_power = df[power_col].mean()
    else:
        expected_power = df['hour'].map(hourly_avg_power).fillna(df[power_col].mean())
    
    df['residual'] = df[power_col] - expected_power
    df['residual_ratio'] = df['residual'] / np.maximum(expected_power, 0.1)  # avoid div by zero
    
    # 15-minute slope
    df['slope_15m'] = df['trailing_15m_delta'] / 0.25  # delta per hour
    
    # === MORNING VS EVENING CUES ===
    # Add date for daily grouping
    df['date'] = df['Timestamp_Local'].dt.date
    
    # Morning baseline (10:45-11:15 average, carried forward for the day)
    # For simplicity, use hour 11 as proxy for morning baseline
    morning_mask = (df['hour'] == 11)
    daily_morning_baseline = df[morning_mask].groupby('date')[power_col].mean()
    
    # Map back to all rows for each date
    df['morning_baseline'] = df['date'].map(daily_morning_baseline).fillna(0)
    df['morning_baseline_available'] = df['date'].isin(daily_morning_baseline.index).astype(int)
    
    # Evening trailing average (17:30-18:00 proxy - use hour 18)
    # For each row, calculate trailing 30-min if in evening window
    df['evening_trailing_30'] = np.where(
        df['hour'] >= 17,
        df[power_col].rolling(window=2, min_periods=1).mean(),
        0
    )
    df['evening_window_active'] = (df['hour'] >= 17).astype(int)
    
    # Morning vs evening delta
    df['evening_delta'] = df['evening_trailing_30'] - df['morning_baseline']
    
    # Binary cue for delta > 8 (soft evidence)
    df['delta_gt_8'] = (df['evening_delta'] > 8).astype(int)
    
    # Fill NaN values from rolling operations
    numeric_cols = ['trailing_15m_delta', 'trailing_30m_delta', 'trailing_60m_avg', 
                   'slope_15m', 'evening_delta']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = df[col].fillna(0)
    
    # Drop temporary date column
    df = df.drop(columns=['date'])
    
    return df

In [ ]:
# Apply enhanced features to all datasets
print("Creating enhanced time-based features for all sites...")

# Calculate hourly averages from Site B training data
hourly_averages = calculate_hourly_averages(df_b)

# Apply enhanced features with calculated hourly averages
df_b_enhanced = create_enhanced_features(df_b, hourly_averages)
df_d_enhanced = create_enhanced_features(df_d, hourly_averages)
df_c_enhanced = create_enhanced_features(df_c, hourly_averages)

print("Adding learnable signal features...")
df_b_signals = create_learnable_signals(df_b_enhanced, hourly_averages)
df_d_signals = create_learnable_signals(df_d_enhanced, hourly_averages)
df_c_signals = create_learnable_signals(df_c_enhanced, hourly_averages)

In [ ]:
# Data Preprocessing and Feature Scaling
# Remove unnecessary columns but KEEP hour and month for better temporal learning
columns_to_drop = ["Site", "Timestamp_Local"]

# For site B, also drop Demand_Response_Capacity_kW if it exists
if "Demand_Response_Capacity_kW" in df_b_signals.columns:
    columns_to_drop.append("Demand_Response_Capacity_kW")

df_b_clean = df_b_signals.drop(columns=[col for col in columns_to_drop if col in df_b_signals.columns])
df_d_clean = df_d_signals.drop(columns=[col for col in columns_to_drop if col in df_d_signals.columns])
df_c_clean = df_c_signals.drop(columns=[col for col in columns_to_drop if col in df_c_signals.columns])

print("Cleaned dataframe columns (keeping hour and month + all learnable signals):")
print("Site B (train):", len(df_b_clean.columns), "columns")
print("Site D (validation):", len(df_d_clean.columns), "columns") 
print("Site C (test):", len(df_c_clean.columns), "columns")

In [ ]:
original_features = ['Dry_Bulb_Temperature_C', 'Global_Horizontal_Radiation_W/m2', 'Building_Power_kW']
temporal_features = ['hour', 'month']  # Numerical temporal features
time_features = ['is_10_to_18','is_10', 'is_11', 'is_12', 'is_13', 'is_14', 'is_15', 'is_16', 'is_17', 'is_january', 'is_february', 'is_june', 'is_july', 'is_august', 'is_december']

power_pattern_features = ['is_morning_peak', 'is_afternoon_low']
power_deviation_features = ['power_deviation', 'power_deviation_ratio']

cyclical_features = ['hour_sin', 'hour_cos', 'month_sin', 'month_cos']
soft_mask_features = ['is_in_business_hours', 'is_allowed_month', 'position_in_window']
causal_delta_features = ['trailing_15m_delta', 'trailing_30m_delta', 'trailing_60m_avg', 'slope_15m']
residual_features = ['residual', 'residual_ratio']
morning_evening_features = ['morning_baseline', 'evening_trailing_30', 'evening_delta', 'delta_gt_8']
availability_features = ['morning_baseline_available', 'evening_window_active']

learnable_signal_features = (cyclical_features + soft_mask_features + causal_delta_features + 
                           residual_features + morning_evening_features + availability_features)

all_binary_features = (time_features + power_pattern_features + soft_mask_features + 
                      morning_evening_features[-1:] + availability_features)  # delta_gt_8 is binary
all_numerical_features = (original_features + temporal_features + power_deviation_features + 
                         cyclical_features + causal_delta_features + residual_features + 
                         morning_evening_features[:-1] + ['position_in_window'])  # exclude delta_gt_8

features = all_numerical_features + all_binary_features

target = "Demand_Response_Flag"

numerical_features_to_scale = all_numerical_features
scaler = MinMaxScaler()
scaled_features_b = scaler.fit_transform(df_b_clean[numerical_features_to_scale])  # Fit on training data (Site B)
scaled_features_d = scaler.transform(df_d_clean[numerical_features_to_scale])  # Transform validation
scaled_features_c = scaler.transform(df_c_clean[numerical_features_to_scale])  # Transform test

print(f"\nScaled features shape: {scaled_features_b.shape}")

# Function to map scaled numerical features back to dataframes
def map_scaled_features(df, numerical_feature_list, scaled_array):
    df_copy = df.copy()
    for i, col in enumerate(numerical_feature_list):
        df_copy[col] = scaled_array[:, i]
    return df_copy

df_b_final = map_scaled_features(df_b_clean, numerical_features_to_scale, scaled_features_b)
df_d_final = map_scaled_features(df_d_clean, numerical_features_to_scale, scaled_features_d)
df_c_final = map_scaled_features(df_c_clean, numerical_features_to_scale, scaled_features_c)

print("\nFinal preprocessed data shapes:")
print("Site B (train):", df_b_final.shape)
print("Site D (validation):", df_d_final.shape)
print("Site C (test):", df_c_final.shape)

In [ ]:
# Create Sequences for LSTM
def create_sequences(df, feature_cols, target_col, seq_len):
    """
    Create sequences for LSTM input with enhanced features and learnable signals
    """
    data = df[feature_cols].values
    labels = df[target_col].values

    X, y = [], []
    for i in range(seq_len, len(df)):
        X.append(data[i-seq_len:i])  # past N steps with all features
        y.append(labels[i])          # current label
    return np.array(X), np.array(y)

# Create sequences with 192 timesteps
seq_length = 192
print(f"Creating sequences with length {seq_length}...")

X_b, y_b = create_sequences(df_b_final, features, target, seq_length)
X_d, y_d = create_sequences(df_d_final, features, target, seq_length)
X_c, y_c = create_sequences(df_c_final, features, target, seq_length)

print("Sequence shapes:")
print(f"Site B (train): X={X_b.shape}, y={y_b.shape}")
print(f"Site D (val):   X={X_d.shape}, y={y_d.shape}")
print(f"Site C (test):  X={X_c.shape}, y={y_c.shape}")

In [ ]:
n_features = X_b.shape[2]
seq_len = X_b.shape[1]

print(f"Building LSTM model with {n_features} features and {seq_len} timesteps...")
print(f"Feature count breakdown: {len(features)} total features")

model = Sequential([
    LSTM(64, input_shape=[seq_len, n_features], return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.1),  # Additional dropout for regularization
    Dense(3, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Model architecture:")
model.summary()

In [ ]:
print("Original label distribution in y_b:", np.unique(y_b, return_counts=True))
print("Original label distribution in y_d:", np.unique(y_d, return_counts=True))
print("Original label distribution in y_c:", np.unique(y_c, return_counts=True))

# Convert labels: -1 -> 0, 0 -> 1, 1 -> 2
y_b_cat = np.where(y_b == -1, 0, np.where(y_b == 0, 1, 2))  # Training labels
y_d_cat = np.where(y_d == -1, 0, np.where(y_d == 0, 1, 2))  # Validation labels
# Note: y_c_cat not needed since Site C flags are all 0 and need to be predicted

print("Converted label distribution:")
print("y_b_cat (train):", np.unique(y_b_cat, return_counts=True))
print("y_d_cat (validation):", np.unique(y_d_cat, return_counts=True))

In [ ]:
# Train the Enhanced LSTM Model
print("Training enhanced LSTM model...")
print(f"Training data: {X_b.shape[0]} samples")
print(f"Validation data: {X_d.shape[0]} samples")

# Add early stopping for better training
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    X_b, y_b_cat,
    validation_data=(X_d, y_d_cat),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

print("Training completed!")

In [ ]:
print(f"Site C has {X_c.shape[0]} sequences to predict")

# Make predictions on Site C (no true labels available)
y_c_pred_prob = model.predict(X_c, verbose=0)
y_c_pred = np.argmax(y_c_pred_prob, axis=1)

# Convert categorical predictions back to original label format (-1, 0, 1)
y_c_pred_original = np.where(y_c_pred == 0, -1, np.where(y_c_pred == 1, 0, 1))

print(f"Site E prediction shapes: {y_c_pred_original.shape}")
print("Site E prediction distribution:", np.unique(y_c_pred_original, return_counts=True))

# Create submission DataFrame in required format
start_idx_c = seq_length  # Account for sequence offset
submission_df = pd.DataFrame()


# Align length with predictions
site_slice = df_c['Site'].iloc[start_idx_c:start_idx_c + len(y_c_pred_original)].copy()
# Fill missing/empty values
fallback_site = site_slice.mode().iloc[0] if not site_slice.mode().empty else 'siteE'
site_slice = site_slice.fillna(fallback_site).replace('', fallback_site)

submission_df['Site'] = site_slice
submission_df['Timestamp_Local'] = df_c['Timestamp_Local'].iloc[start_idx_c:start_idx_c + len(y_c_pred_original)].values
submission_df['Demand_Response_Flag'] = y_c_pred_original
submission_df['Demand_Response_Capacity_kW'] = 0


print(f"Submission DataFrame created with shape: {submission_df.shape}")
print(f"Columns: {list(submission_df.columns)}")

submission_path = ''
submission_df.to_csv(submission_path, index=False)
print(f"Competition submission file saved to: {submission_path}")

# Show prediction summary
print(f"Total predictions: {len(y_c_pred_original)}")
print(f"Flag -1 (Negative DR): {np.sum(y_c_pred_original == -1)} ({np.sum(y_c_pred_original == -1)/len(y_c_pred_original)*100:.1f}%)")
print(f"Flag  0 (No event):    {np.sum(y_c_pred_original == 0)} ({np.sum(y_c_pred_original == 0)/len(y_c_pred_original)*100:.1f}%)")
print(f"Flag  1 (Positive DR): {np.sum(y_c_pred_original == 1)} ({np.sum(y_c_pred_original == 1)/len(y_c_pred_original)*100:.1f}%)")

In [ ]:
# Create directory for saving model artifacts
import os
save_dir = ''
os.makedirs(save_dir, exist_ok=True)

# 1. Save the trained model
model_path = os.path.join(save_dir, 'lstm(best)_model.h5')
model.save(model_path)
print(f"Model saved to: {model_path}")

# 2. Save the scaler
scaler_path = os.path.join(save_dir, 'lstm(best)_scaler.pkl')
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f"Scaler saved to: {scaler_path}")

# 4. Save feature lists and model configuration
config = {
    'features': features,
    'numerical_features_to_scale': numerical_features_to_scale,
    'all_binary_features': all_binary_features,
    'all_numerical_features': all_numerical_features,
    'seq_length': seq_length,
    'n_features': n_features,
    'target': target,
    'original_features': original_features,
    'temporal_features': temporal_features,
    'time_features': time_features,
    'power_pattern_features': power_pattern_features,
    'power_deviation_features': power_deviation_features,
    'cyclical_features': cyclical_features,
    'soft_mask_features': soft_mask_features,
    'causal_delta_features': causal_delta_features,
    'residual_features': residual_features,
    'morning_evening_features': morning_evening_features,
    'availability_features': availability_features,
    'learnable_signal_features': learnable_signal_features
}

config_path = os.path.join(save_dir, 'lstm_model(best)_config.json')
import json
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)
print(f"Model configuration saved to: {config_path}")

# 5. Save training history
history_path = os.path.join(save_dir, 'training(best)_history.pkl')
with open(history_path, 'wb') as f:
    pickle.dump(history.history, f)
print(f"Training history saved to: {history_path}")

print(f"\nAll model artifacts saved in directory: {save_dir}")
print("Contents:")
for file in os.listdir(save_dir):
    print(f"  - {file}")